# Устанавливаем необходимые модули

Модуль определения координат

In [ ]:
!pip install geocoder

зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:

In [ ]:
!pip freeze > requirements.txt 

# Импортируем необходимые библиотеки

In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# импортируем библиотеки для определения геопозиции
import geocoder

# импорт регулярок
import re

# импорт модуля управления датами и временем
from datetime import datetime as dt
import datetime

# импортируем контейнер типов данных
import collections

# импорт модуля кодирования категориальных значений
import category_encoders as ce

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Задаем параметры

In [28]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

# Загрузим данные из соревнования

In [29]:
df_train = pd.read_csv('data/hotels_test.csv.zip') # датасет для обучения
df_test = pd.read_csv('data/hotels_test.csv.zip') # датасет для предсказания
sample_submission = pd.read_csv('data/submission.csv.zip') # самбмишн

In [ ]:
df_train.info() # смотрим состав датасета для обучения

In [ ]:
df_train.head(2) # знакомимся с данными датасета для обучения

In [ ]:
df_test.info() # смотрим состав датасета для предсказания

In [ ]:
df_test.head(2) # знакомимся с данными датасета для предсказания

In [ ]:
sample_submission.head(2) # знакомимся с данными датасета самбмишн

In [ ]:
sample_submission.info() # смотрим состав датасета самбмишн

# Поиск дублей

In [36]:
mask = df_train.duplicated(subset=df_train.columns) # маска для фильтрации
data_duplicates = df_train[mask] # фильтруем наш датасет
print(f'Число найденных дубликатов: {data_duplicates.shape[0]}')

Число найденных дубликатов: 29


In [37]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info() # смотрим состав объединенного датасета

In [ ]:
# Преобразуем строковое значение количества дней между датой проверки и датой очистки в числовое
data['days_since_review'] = data['days_since_review'].apply(lambda value: np.int64(re.search('\d+', value).group(0)))
data['days_since_review']

# Заполняем пропуски в данных координат в DataFrame 'data'

In [40]:
# Получаем список адресов отелей с координатами
hotels_df = data[['hotel_address','lat','lng']].groupby(by=['hotel_address'],as_index=True).mean()

# Определяем недостающие координаты отелей
for i in hotels_df[hotels_df['lat'].isnull()].index:
    g =geocoder.arcgis(i) # Делаем запрос координат через модуль geocoder
    hotels_df.at[i,'lat'] = g.latlng[0]
    hotels_df.at[i,'lng'] = g.latlng[1]

# Разбираем 'addres' на страну и адрес
hotels_df['country'] = ''
hotels_df['addres'] = hotels_df.index
hotels_df['country'] = hotels_df['addres'].apply(lambda x: x.split(' ')[-1] if (x.split(' ')[-1] != 'Kingdom') else (x.split(' ')[-2]+' '+x.split(' ')[-1]))

# Преобразуем данные в словарь
coord_dict = hotels_df.to_dict(orient='index')
    
# Заполняем координаты в DataFrame'е
for i in data[data['lat'].isnull()].index:
    data.at[i,'lat'] = coord_dict[data['hotel_address'][i]]['lat']
    data.at[i,'lng'] = coord_dict[data['hotel_address'][i]]['lng']
        
# Определяем являются ли отели сетевыми
data['hotel_net'] = data['hotel_name'].apply(
    lambda x: re.search(r'Mercure|Golden Tulip|Holiday Inn|Golden Tulip|Radisson|Best Western|DoubleTree',x)[0] \
    if (re.search(r'Mercure|Golden Tulip|Holiday Inn|Golden Tulip|Radisson|Best Western|DoubleTree',x) is not None) \
    else 'Not_in_net'
    )
    
# Выделяем страну в отдельный столбец
data['country'] = ''
data['country'] = data['hotel_address'].apply(lambda x: x.split(' ')[-1] if (x.split(' ')[-1] != 'Kingdom') else (x.split(' ')[-2]+' '+x.split(' ')[-1]))

# Исправляем представление данных о национальности
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x[1:-1])

In [ ]:
data.head(2) # убеждаемся в изменениях датасета  

In [ ]:
data.info()

# Добавляем данные: Город(где находится отель), площадь и популяция  

In [ ]:
 
data['hotel_city'] = data['hotel_address'].apply(lambda x: 'London' if x.endswith('United Kingdom') else x.split()[-2])

сity_population = {
    'Paris':  2148327, 'London': 8908081, 'Milan': 1366180, 
    'Vienna': 1911191, 'Barcelona': 1636732, 'Amsterdam': 860124
}
сity_area = {
    'Paris': 105, 'London': 1706, 'Milan': 181, 
    'Vienna': 414, 'Barcelona': 101, 'Amsterdam': 219
}

data['сity_population'] = data['hotel_city'].map(сity_population)
data['сity_area'] = data['hotel_city'].map(сity_area)

data.drop(['hotel_address'], axis=1, inplace=True)

data.info()

In [ ]:
data['hotel_city'] # Посмотрим на столбец с названием городов

# Посчитаем количество дней с момента опубликования ревью

In [ ]:
data['review_date'] = pd.to_datetime(data['review_date'])
now_date = datetime.date.today()

date_1=pd.to_datetime(now_date)
print(date_1)
data['review_date']

In [48]:
# рассчитаем количество дней прошедших с ревью
now_date = datetime.date.today()
data['days_with_a_review'] = (date_1 - data['review_date']).dt.days

In [ ]:
data.info()

In [51]:
# Преобразуем дату ревью в признаки
data['year'] = data['review_date'].dt.year
data['month'] = data['review_date'].dt.month
data['day'] = data['review_date'].dt.day
data['dayofweek'] = data['review_date'].dt.dayofweek

In [52]:
# Удаляем столбец с датами
data.drop('review_date', axis = 1, inplace=True)

# Корректировка информации о национальности ревьюера. Также мы преобразуем национальность в признак

In [ ]:
# Поправим данные о ревьюере из США
us_list = ['Jersey','United States of America']
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: 'United States' if x in us_list else x)

# Преобразуем национальность ревьюера в признак
nat_list = []
for i in data.index:
    nat_list.append(data['reviewer_nationality'][i]) 
nat_cnt = collections.Counter(nat_list)

top_nat_set = set([key for key, _ in nat_cnt.most_common(5)])
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x if x in top_nat_set else 'other')
data['reviewer_nationality'].describe()

In [ ]:
data.info()

In [ ]:
data.nunique(dropna=False)

# Кодировка признаков

Сделаем кодировку признаков 'hotel_city', 'hotel_net' и 'country' методом однократного кодирования (так как количество значений незначительное). Исходный признак удаляем.

In [ ]:
encoder = ce.OneHotEncoder(cols=['hotel_city'])
type_bin_city = encoder.fit_transform(data['hotel_city'])
data = pd.concat([data, type_bin_city], axis=1)

encoder = ce.OneHotEncoder(cols=['hotel_net'])
type_bin_net = encoder.fit_transform(data['hotel_net'])
data = pd.concat([data, type_bin_net], axis=1)

encoder = ce.OneHotEncoder(cols=['country'])
type_bin_country = encoder.fit_transform(data['country'])
data = pd.concat([data, type_bin_country], axis=1)

data.drop(['hotel_city', 'hotel_net', 'country'], axis=1, inplace=True)
data.info()


Сделаем кодировку признаков 'reviewer_nationality' и 'hotel_name' методом двоичного кодирования (так как количество значений более 200). Исходный признак удаляем.

In [ ]:
bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality'])
type_bin = bin_encoder.fit_transform(data['reviewer_nationality'])
data = pd.concat([data, type_bin], axis=1)

bin_encoder = ce.BinaryEncoder(cols=['hotel_name'])
type_bin = bin_encoder.fit_transform(data['hotel_name'])
data = pd.concat([data, type_bin], axis=1)

data.drop(['reviewer_nationality', 'hotel_name'], axis=1, inplace=True)
data.info()

# Посмотрим на корреляцию получившихся признаков

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

ввиду того что наблюдается корреляция между признаками 'country', 'city', 'сity_area' и 'days_since_review', удаляем лишние признаки 

In [59]:
data.drop(['country_1', 'country_2', 'country_3', 'country_4', 'country_5', 'country_6', 'days_since_review', 'сity_area'], axis=1, inplace=True)

# Проверим корреляцию

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

In [61]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
data.info()

In [63]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [74]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [75]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

((128935, 42), (128935, 42), (128935, 41), (103148, 41), (25787, 41))

In [76]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [77]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [393]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [394]:
test_data.sample(10)

In [395]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [396]:
sample_submission

In [397]:
predict_submission = model.predict(test_data)

In [398]:
predict_submission

In [399]:
list(sample_submission)

In [400]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)